**Dataset:** Regression with a tabular California housing dataset [Data set].
Kaggle. https://kaggle.com/competitions/playground-series-s3e1

**Architecture:** 

In [12]:
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
url_train = 'https://raw.githubusercontent.com/dmitryglhf/fedcore_lossreg/refs/heads/main/data/kaggle_s3e1/train.csv'
url_test = 'https://raw.githubusercontent.com/dmitryglhf/fedcore_lossreg/refs/heads/main/data/kaggle_s3e1/test.csv'

train = pd.read_csv(url_train, index_col=0)
test = pd.read_csv(url_test, index_col=0)

In [14]:
train_tensor = torch.tensor(train.values, dtype=torch.float32)
test_tensor = torch.tensor(test.values, dtype=torch.float32)

In [15]:
train_loader = DataLoader(train_tensor, batch_size=32, shuffle=True)
test_loader = DataLoader(test_tensor, batch_size=32)